In [1]:
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy
import pandas as pd

In [2]:
reader = Reader()

In [3]:
# data_folder = join('..', 'data')
# dir_list = [str(x) for x in range(1, 11)]
# dir_list = ['1', '2', '3', '5', '6', '7', '8', '9', '10']

In [4]:
data_folder = join('..', 'data_chicken')
dir_list = ['7', '8', '9', '10']

In [5]:
#blacklist = ['1/run2', '3/run3', '3/run5']
blacklist = ['7/run2']
blacklist = [join(data_folder, x) for x in blacklist]

In [25]:
specialty_keys = ["neuro", "plastic"]
expertise_keys = ["novice", "expert", "intermediate1", "intermediate2"]

In [26]:
knot_time = {}
tps_forces = {}
emg_data = {}
tps_data_type = 'raw'

for d in dir_list:
    knots_dir = [x[0] for x in os.walk(join(data_folder, d))][1:]
    knots_dir.sort()
    knot_time[d] = []
    tps_forces[d] = {}
    emg_data[d] = {}
    
    for l in ['right', 'left']:
        for f in ['index', 'middle', 'thumb']:
            key = l + '_' + f + '_' + tps_data_type
            tps_forces[d][key] = []
            
    for l in ['emg' + str(x) for x in range(8)]:
         emg_data[d][l] = []
        
    for kd in knots_dir:
        if not kd in blacklist:
            print(kd)
            reader.play(kd)
            time = reader.data['camera']['relative_time'].iloc[-1]
            knot_time[d].append(time)
            for l in ['right', 'left']:
                for f in ['index', 'middle', 'thumb']:
                    key = l + '_' + f + '_' + tps_data_type
                    tps_forces[d][key].append(reader.data['tps'][key].to_numpy())
            for l in ['emg' + str(x) for x in range(8)]:
                emg_data[d][l].append(np.absolute(reader.data['emg'][l].to_numpy()))

../data_chicken/7/calibration
../data_chicken/7/run
../data_chicken/7/run3
../data_chicken/8/run1
../data_chicken/9/run1
../data_chicken/10/run1
../data_chicken/10/run2
../data_chicken/10/run3


In [27]:
subject_data = pd.read_csv(join(data_folder,"subject_data.csv"))

In [28]:
subject_data

,ID,Sex,Age,Laterality,Specialty,Expertise,Ali score,Score
0,1,M,37,R,plastic,intermediate,5,estimated
1,2,F,30,R,plastic,novice,10,estimated
2,3,M,41,R,plastic,expert,5,estimated
3,4,M,29,L,neuro,novice,10,estimated
4,5,F,34,R,neuro,novice,8,estimated
5,6,M,45,R,neuro,expert,3,real
6,7,M,36,R,neuro,novice,10,real
7,8,M,45,R,plastic,expert,7,real
8,9,M,34,R,neuro,intermediate1,6,real
9,10,M,33,L,neuro,intermediate2,6,real


# Execution time

In [29]:
for key, value in knot_time.items():
    print(key, np.mean(value), np.std(value))

7 335.4441443284353 236.5085727254296
8 597.5771343708038 0.0
9 1135.5206537246704 0.0
10 520.0086479187012 43.76653734505754


## Differences between subjects

In [30]:
idx = 0
for subject_id1 in dir_list:
    tmp =  dir_list[idx + 1:]
    for subject_id2 in tmp:
        print('diff between ' + subject_id1 + ' and ' + subject_id2)
        print(stats.ttest_ind(knot_time[subject_id1], knot_time[subject_id2]))
        print('--------------')
    idx = idx + 1

diff between 7 and 8
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between 7 and 9
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between 7 and 10
Ttest_indResult(statistic=-1.0851872518698396, pvalue=0.3388665316779968)
--------------
diff between 8 and 9
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between 8 and 10
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between 9 and 10
Ttest_indResult(statistic=nan, pvalue=nan)
--------------


/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3584: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars



# TPS Data

In [31]:
average_tps_per_run = {}
for key, value in tps_forces.items():
    average_tps_per_run[key] = {}
    for finger, values in value.items():
        average_tps_per_run[key][finger] = []
        for run in values:
            average_tps_per_run[key][finger].append(np.mean(run))

In [32]:
average_tps = {}
for subject, fingers in average_tps_per_run.items():
    average_tps[subject] = {}
    for finger, values in fingers.items():
        average_tps[subject][finger] = {}
        average_tps[subject][finger]['mean'] = np.mean(values)
        average_tps[subject][finger]['std'] = np.std(values)

In [33]:
fig = go.Figure()
for subject, fingers in average_tps.items():
    finger_names = list(fingers.keys())
    finger_mean = []
    finger_std = []
    for finger in finger_names:
        finger_mean.append(fingers[finger]['mean'])
        finger_std.append(fingers[finger]['std'])
    fig.add_trace(go.Bar(
        name=subject,
        x=finger_names,
        y=finger_mean,
        error_y=dict(type='data', array=finger_std)
    ))
fig.update_layout(barmode='group')

### group per specialty

In [34]:
specialty_tps = {}
for s in specialty_keys:
    specialty_tps[s] = {}

for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f
        for s in specialty_keys:
            specialty_tps[s][key] = []
            
for key, value in tps_forces.items():
    specialty = subject_data.loc[subject_data["ID"] == int(key)]["Specialty"].values[0]
    for finger, values in value.items(): 
        f = finger.replace("_raw", "")
        for run in values:
            specialty_tps[specialty][f] = specialty_tps[specialty][f] + run.tolist()

In [35]:
average_specialty_tps = {}
for s in specialty_keys:
    average_specialty_tps[s] = {}

for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f
        for s in specialty_keys:
            average_specialty_tps[s][key] = {}
        
for specialty, data in specialty_tps.items():
    for finger, values in data.items():
        average_specialty_tps[specialty][finger]["mean"] = np.mean(values)
        average_specialty_tps[specialty][finger]["std"] = np.std(values)

In [36]:
fig = go.Figure()
for specialty, fingers in average_specialty_tps.items():
    finger_names = list(fingers.keys())
    finger_mean = []
    finger_std = []
    for finger in finger_names:
        finger_mean.append(fingers[finger]['mean'])
        finger_std.append(fingers[finger]['std'])
    fig.add_trace(go.Bar(
        name=specialty,
        x=finger_names,
        y=finger_mean,
        error_y=dict(type='data', array=finger_std)
    ))
fig.update_layout(barmode='group')
fig.write_image(join(data_folder, "figures", "tps_per_speciatly.svg"))

### heatmap per specialty

In [37]:
for specialty in specialty_keys:
    x_values = []
    y_values = []
    y_min = 8000
    y_max = 16000

    fig = go.Figure()
    for i, finger in enumerate(average_specialty_tps[specialty].keys()):
        x_values.append(i)
        y_values.append(average_specialty_tps[specialty][finger]["mean"])
    y_values = (np.array(y_values) - 8000) / 8000

    fig.add_trace(go.Scatter(
        x=x_values,
        y=y_values,
        marker=dict(
            size=16,
            cmax=1,
            cmin=0,
            color=y_values,
            colorbar=dict(
                title="Colorbar"
            ),
            colorscale="Turbo"
        ),
        mode="markers"))

    fig.write_image(join(data_folder, "figures", "colorscale_tps_" + specialty + ".svg"))

### group per expertise

In [38]:
expertise_tps = {}
for e in expertise_keys:
    expertise_tps[e] = {}

for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f
        for e in expertise_keys:
            expertise_tps[e][key] = []
            
for key, value in tps_forces.items():
    expertise = subject_data.loc[subject_data["ID"] == int(key)]["Expertise"].values[0]
    for finger, values in value.items(): 
        f = finger.replace("_raw", "")
        for run in values:
            expertise_tps[expertise][f] = expertise_tps[expertise][f] + run.tolist()

In [39]:
average_expertise_tps = {}
for e in expertise_keys:
    average_expertise_tps[e] = {}

for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f
        for e in expertise_keys:
            average_expertise_tps[e][key] = {}
        
for expertise, data in expertise_tps.items():
    for finger, values in data.items():
        average_expertise_tps[expertise][finger]["mean"] = np.mean(values)
        average_expertise_tps[expertise][finger]["std"] = np.std(values)

In [40]:
fig = go.Figure()
for expertise, fingers in average_expertise_tps.items():
    finger_names = list(fingers.keys())
    finger_mean = []
    finger_std = []
    for finger in finger_names:
        finger_mean.append(fingers[finger]['mean'])
        finger_std.append(fingers[finger]['std'])
    fig.add_trace(go.Bar(
        name=expertise,
        x=finger_names,
        y=finger_mean,
        error_y=dict(type='data', array=finger_std)
    ))
fig.update_layout(barmode='group')
fig.write_image(join(data_folder, "figures", "tps_per_expertise.svg"))

### heatmap per expertise

In [41]:
for expertise in expertise_keys:
    x_values = []
    y_values = []
    y_min = 8000
    y_max = 16000

    fig = go.Figure()
    for i, finger in enumerate(average_expertise_tps[expertise].keys()):
        x_values.append(i)
        y_values.append(average_expertise_tps[expertise][finger]["mean"])
    y_values = (np.array(y_values) - 8000) / 8000

    fig.add_trace(go.Scatter(
        x=x_values,
        y=y_values,
        marker=dict(
            size=16,
            cmax=1,
            cmin=0,
            color=y_values,
            colorbar=dict(
                title="Colorbar"
            ),
            colorscale="Turbo"
        ),
        mode="markers"))

    fig.write_image(join(data_folder, "figures", "colorscale_tps_" + expertise + ".svg"))

### group per specialty & expertise

In [42]:
specialty_expertise_tps = {}
for s in specialty_keys:
    specialty_expertise_tps[s] = {}
    for expertise in expertise_keys:
        specialty_expertise_tps[s][expertise] = {}

for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f
        for s in specialty_keys:
            for expertise in expertise_keys:
                specialty_expertise_tps[s][expertise][key] = []
            
for key, value in tps_forces.items():
    specialty = subject_data.loc[subject_data["ID"] == int(key)]["Specialty"].values[0]
    expertise = subject_data.loc[subject_data["ID"] == int(key)]["Expertise"].values[0]
    for finger, values in value.items(): 
        f = finger.replace("_raw", "")
        for run in values:
            specialty_expertise_tps[specialty][expertise][f] = specialty_expertise_tps[specialty][expertise][f] + run.tolist()

In [43]:
average_specialty_expertise_tps = {}
for s in specialty_keys:
    average_specialty_expertise_tps[s] = {}
    for expertise in expertise_keys:
        average_specialty_expertise_tps[s][expertise] = {}
        average_specialty_expertise_tps[s][expertise] = {}

for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f
        for s in specialty_keys:
            for expertise in expertise_keys:
                average_specialty_expertise_tps[s][expertise][key] = {}
        
for specialty, data in specialty_expertise_tps.items():
    for expertise, fingers in data.items():
        for finger, values in fingers.items():
            average_specialty_expertise_tps[specialty][expertise][finger]["mean"] = np.mean(values)
            average_specialty_expertise_tps[specialty][expertise][finger]["std"] = np.std(values)

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning:

invalid value encountered in true_divide



In [44]:
for specialty in specialty_keys:
    fig = go.Figure()
    for expertise, fingers in average_specialty_expertise_tps[specialty].items():
        finger_names = list(fingers.keys())
        finger_mean = []
        finger_std = []
        for finger in finger_names:
            finger_mean.append(fingers[finger]['mean'])
            finger_std.append(fingers[finger]['std'])
        fig.add_trace(go.Bar(
            name=expertise,
            x=finger_names,
            y=finger_mean,
            error_y=dict(type='data', array=finger_std)
        ))
    fig.update_layout(barmode='group')
    fig.write_image(join(data_folder, "figures", "tps_per_expertise_" + specialty + ".svg"))

### two way ANOVA

In [45]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd

In [46]:
# create the data separating expertise and specialty for each fingers
finger_data = {}
for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f + '_raw'
        finger_data[key] = []
    
for subject, data in average_tps.items():
    specialty = subject_data.loc[subject_data["ID"] == int(subject)]["Specialty"].values[0]
    expertise = subject_data.loc[subject_data["ID"] == int(subject)]["Expertise"].values[0]
    for finger, values in data.items():
        row = [specialty, expertise, values["mean"]]
        finger_data[finger].append(row)
                
finger_dataframes = {}
for finger, values in finger_data.items():
    finger_dataframes[finger] = pd.DataFrame(values, columns=["specialty", "expertise", "value"])

In [47]:
finger_anova_results = {}
for finger, dataframe in finger_dataframes.items():
    #perform two-way ANOVA
    model = ols('value ~ C(specialty) + C(expertise) + C(specialty):C(expertise)', data=dataframe).fit()
    finger_anova_results[finger] = sm.stats.anova_lm(model, typ=2)

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1650: RuntimeWarning:

divide by zero encountered in double_scalars

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/statsmodels/base/model.py:1452: RuntimeWarning:

invalid value encountered in multiply



ValueError: array must not contain infs or NaNs

In [48]:
finger_anova_results["right_index_raw"]

KeyError: 'right_index_raw'

In [49]:
finger_anova_results["left_index_raw"]

KeyError: 'left_index_raw'

In [50]:
finger_anova_results["right_middle_raw"]

KeyError: 'right_middle_raw'

In [51]:
finger_anova_results["left_middle_raw"]

KeyError: 'left_middle_raw'

In [52]:
finger_anova_results["right_thumb_raw"]

KeyError: 'right_thumb_raw'

In [53]:
finger_anova_results["left_thumb_raw"]

KeyError: 'left_thumb_raw'

### t-test

In [54]:
expertise_finger_data = {}
specialty_finger_data = {}

for l in ['right', 'left']:
    for f in ['index', 'middle', 'thumb']:
        key = l + '_' + f + '_raw'
        expertise_finger_data[key] = {}
        specialty_finger_data[key] = {}
        for specialty in specialty_keys:
            specialty_finger_data[key][specialty] = []
        for expertise in expertise_keys:
            expertise_finger_data[key][expertise] = []
        
for subject, data in average_tps_per_run.items():
    specialty = subject_data.loc[subject_data["ID"] == int(subject)]["Specialty"].values[0]
    expertise = subject_data.loc[subject_data["ID"] == int(subject)]["Expertise"].values[0]
    for finger, values in data.items():
        expertise_finger_data[finger][expertise] = expertise_finger_data[finger][expertise] + values
        specialty_finger_data[finger][specialty] = specialty_finger_data[finger][specialty] + values

In [55]:
for f, data in expertise_finger_data.items():
    print('####### ' + f + ' ########')
    keys = list(data.keys())
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            print('diff between ' + keys[i] + ' and ' + keys[j])
            print(stats.ttest_ind(data[keys[i]], data[keys[j]]))
            print('--------------')

####### right_index_raw ########
diff between novice and expert
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between novice and intermediate1
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between novice and intermediate2
Ttest_indResult(statistic=-6.830332035808608, pvalue=0.0024029207893389386)
--------------
diff between expert and intermediate1
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between expert and intermediate2
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between intermediate1 and intermediate2
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
####### right_middle_raw ########
diff between novice and expert
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between novice and intermediate1
Ttest_indResult(statistic=nan, pvalue=nan)
--------------
diff between novice and intermediate2
Ttest_indResult(statistic=0.6956990466488792, pvalue=0.5249205657514293)
--------------
diff between 

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3584: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars



# EMG analysis

In [56]:
average_emg_per_run = {}
for key, value in emg_data.items():
    average_emg_per_run[key] = {}
    for channel, values in value.items():
        average_emg_per_run[key][channel] = []
        for run in values:
            mean_value = np.mean(run)
            if mean_value < 1000:
                average_emg_per_run[key][channel].append(np.mean(run))

In [57]:
average_emg = {}
for subject, channels in average_emg_per_run.items():
    average_emg[subject] = {}
    for channel, values in channels.items():
        average_emg[subject][channel] = {}
        average_emg[subject][channel]['mean'] = np.mean(values)
        average_emg[subject][channel]['std'] = np.std(values)

In [58]:
fig = go.Figure()
for subject, channels in average_emg.items():
    channel_names = list(channels.keys())
    channel_mean = []
    channel_std = []
    for channel in channel_names:
        channel_mean.append(channels[channel]['mean'])
        channel_std.append(channels[channel]['std'])
    fig.add_trace(go.Bar(
        name=subject,
        x=channel_names,
        y=channel_mean,
        error_y=dict(type='data', array=channel_std)
    ))
fig.update_layout(barmode='group')

### group per specialty

In [59]:
specialty_emg = {}
for s in specialty_keys:
    specialty_emg[s] = {}

for key in ['emg' + str(x) for x in range(8)]:
    for s in specialty_keys:
        specialty_emg[s][key] = []
            
for key, value in emg_data.items():
    specialty = subject_data.loc[subject_data["ID"] == int(key)]["Specialty"].values[0]
    for channel, values in value.items(): 
        for run in values:
            mean_value = np.mean(run)
            if mean_value < 1000:
                specialty_emg[specialty][channel] = specialty_emg[specialty][channel] + run.tolist()

In [60]:
average_specialty_emg = {}
for s in specialty_keys:
    average_specialty_emg[s] = {}

for key in ['emg' + str(x) for x in range(8)]:
    for s in specialty_keys:
        average_specialty_emg[s][key] = {}
        
for specialty, data in specialty_emg.items():
    for channel, values in data.items():
        average_specialty_emg[specialty][channel]["mean"] = np.mean(values)
        average_specialty_emg[specialty][channel]["std"] = np.std(values)

In [61]:
fig = go.Figure()
for specialty, channels in average_specialty_emg.items():
    channel_names = list(channels.keys())
    channel_mean = []
    channel_std = []
    for channel in channel_names:
        channel_mean.append(channels[channel]['mean'])
        channel_std.append(channels[channel]['std'])
    fig.add_trace(go.Bar(
        name=specialty,
        x=channel_names,
        y=channel_mean,
        error_y=dict(type='data', array=channel_std)
    ))
fig.update_layout(barmode='group')
fig.write_image(join(data_folder, "figures", "emg_per_speciatly.svg"))

### heatmap per specialty

In [62]:
for specialty in specialty_keys:
    x_values = []
    y_values = []
    y_min = 0
    y_max = 100

    fig = go.Figure()
    for i, channel in enumerate(average_specialty_emg[specialty].keys()):
        x_values.append(i)
        y_values.append(average_specialty_emg[specialty][channel]["mean"])
    y_values = (np.array(y_values) - y_min) / (y_max - y_min)

    fig.add_trace(go.Scatter(
        x=x_values,
        y=y_values,
        marker=dict(
            size=16,
            cmax=1,
            cmin=0,
            color=y_values,
            colorbar=dict(
                title="Colorbar"
            ),
            colorscale="Turbo"
        ),
        mode="markers"))

    fig.write_image(join(data_folder, "figures", "colorscale_emg_" + specialty + ".svg"))

### group per expertise

In [63]:
expertise_emg = {}
for e in expertise_keys:
    expertise_emg[e] = {}

for key in ['emg' + str(x) for x in range(8)]:
    for e in expertise_keys:
        expertise_emg[e][key] = []
            
for key, value in emg_data.items():
    expertise = subject_data.loc[subject_data["ID"] == int(key)]["Expertise"].values[0]
    for channel, values in value.items(): 
        for run in values:
            mean_value = np.mean(run)
            if mean_value < 1000:
                expertise_emg[expertise][channel] = expertise_emg[expertise][channel] + run.tolist()

In [64]:
average_expertise_emg = {}
for e in expertise_keys:
    average_expertise_emg[e] = {}

for key in ['emg' + str(x) for x in range(8)]:
    for e in expertise_keys:
        average_expertise_emg[e][key] = {}
        
for expertise, data in expertise_emg.items():
    for channel, values in data.items():
        average_expertise_emg[expertise][channel]["mean"] = np.mean(values)
        average_expertise_emg[expertise][channel]["std"] = np.std(values)

In [65]:
fig = go.Figure()
for expertise, channels in average_expertise_emg.items():
    channel_names = list(channels.keys())
    channel_mean = []
    channel_std = []
    for channel in channel_names:
        channel_mean.append(channels[channel]['mean'])
        channel_std.append(channels[channel]['std'])
    fig.add_trace(go.Bar(
        name=expertise,
        x=channel_names,
        y=channel_mean,
        error_y=dict(type='data', array=channel_std)
    ))
fig.update_layout(barmode='group')
fig.write_image(join(data_folder, "figures", "emg_per_expertise.svg"))

### heatmap per expertise

In [66]:
for expertise in expertise_keys:
    x_values = []
    y_values = []
    y_min = 0
    y_max = 100

    fig = go.Figure()
    for i, channel in enumerate(average_expertise_emg[expertise].keys()):
        x_values.append(i)
        y_values.append(average_expertise_emg[expertise][channel]["mean"])
    y_values = (np.array(y_values) - y_min) / (y_max - y_min)

    fig.add_trace(go.Scatter(
        x=x_values,
        y=y_values,
        marker=dict(
            size=16,
            cmax=1,
            cmin=0,
            color=y_values,
            colorbar=dict(
                title="Colorbar"
            ),
            colorscale="Turbo"
        ),
        mode="markers"))

    fig.write_image(join(data_folder, "figures", "colorscale_emg_" + expertise + ".svg"))

### group per expertise & specialty

In [67]:
specialty_expertise_emg = {}
for s in specialty_keys:
    specialty_expertise_emg[s] = {}
    for expertise in expertise_keys:
        specialty_expertise_emg[s][expertise] = {}
        for key in ['emg' + str(x) for x in range(8)]:
            specialty_expertise_emg[s][expertise][key] = []
            
for key, value in emg_data.items():
    specialty = subject_data.loc[subject_data["ID"] == int(key)]["Specialty"].values[0]
    expertise = subject_data.loc[subject_data["ID"] == int(key)]["Expertise"].values[0]
    for channel, values in value.items(): 
        for run in values:
            mean_value = np.mean(run)
            if mean_value < 1000:
                specialty_expertise_emg[specialty][expertise][channel] = specialty_expertise_emg[specialty][expertise][channel] + run.tolist()

In [68]:
average_specialty_expertise_emg = {}
for s in specialty_keys:
    average_specialty_expertise_emg[s] = {}
    for expertise in expertise_keys:
        average_specialty_expertise_emg[s][expertise] = {}
        for key in ['emg' + str(x) for x in range(8)]:
            average_specialty_expertise_emg[s][expertise][key] = {}
        
for specialty, data in specialty_expertise_emg.items():
    for expertise, channels in data.items():
        for channel, values in channels.items():
            average_specialty_expertise_emg[specialty][expertise][channel]["mean"] = np.mean(values)
            average_specialty_expertise_emg[specialty][expertise][channel]["std"] = np.std(values)

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/buschbapti/anaconda3/envs/surgeon_recording/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning:

invalid value encountered in true_divide



In [69]:
for specialty in specialty_keys:
    fig = go.Figure()
    for expertise, channels in average_specialty_expertise_emg[specialty].items():
        channel_names = list(channels.keys())
        channel_mean = []
        channel_std = []
        for channel in channel_names:
            channel_mean.append(channels[channel]['mean'])
            channel_std.append(channels[channel]['std'])
        fig.add_trace(go.Bar(
            name=expertise,
            x=channel_names,
            y=channel_mean,
            error_y=dict(type='data', array=channel_std)
        ))
    fig.update_layout(barmode='group')
    fig.write_image(join(data_folder, "figures", "emg_per_expertise_" + specialty + ".svg"))

### two ways ANOVA

In [70]:
# create the data separating expertise and specialty for each channels
channels_data = {}

for key in ['emg' + str(x) for x in range(8)]:
    channels_data[key] = []
        
for subject, data in average_emg.items():
    specialty = subject_data.loc[subject_data["ID"] == int(subject)]["Specialty"].values[0]
    expertise = subject_data.loc[subject_data["ID"] == int(subject)]["Expertise"].values[0]
    for channel, values in data.items():
        row = [specialty, expertise, values["mean"]]
        channels_data[channel].append(row)
                
channels_dataframes = {}
for channel, values in channels_data.items():
    channels_dataframes[channel] = pd.DataFrame(values, columns=["specialty", "expertise", "value"])

In [71]:
emg_anova_results = {}
for channel, dataframe in channels_dataframes.items():
    #perform two-way ANOVA
    model = ols('value ~ C(specialty) + C(expertise) + C(specialty):C(expertise)', data=dataframe).fit()
    emg_anova_results[channel] = sm.stats.anova_lm(model, typ=2)

ValueError: array must not contain infs or NaNs

In [ ]:
emg_anova_results["emg0"]

In [ ]:
emg_anova_results["emg1"]

In [ ]:
emg_anova_results["emg2"]

In [ ]:
emg_anova_results["emg3"]

In [ ]:
emg_anova_results["emg4"]

In [ ]:
emg_anova_results["emg5"]

In [ ]:
emg_anova_results["emg6"]

In [ ]:
emg_anova_results["emg7"]

### t-test

In [ ]:
expertise_channel_data = {}
specialty_channel_data = {}

for key in ['emg' + str(x) for x in range(8)]:
    expertise_channel_data[key] = {}
    specialty_channel_data[key] = {}
    for specialty in specialty_keys:
        specialty_channel_data[key][specialty] = []
    for expertise in expertise_keys:
        expertise_channel_data[key][expertise] = []
        
for subject, data in average_emg_per_run.items():
    specialty = subject_data.loc[subject_data["ID"] == int(subject)]["Specialty"].values[0]
    expertise = subject_data.loc[subject_data["ID"] == int(subject)]["Expertise"].values[0]
    for channel, values in data.items():
        expertise_channel_data[channel][expertise] = expertise_channel_data[channel][expertise] + values
        specialty_channel_data[channel][specialty] = specialty_channel_data[channel][specialty] + values

In [ ]:
for f, data in expertise_channel_data.items():
    print('####### ' + f + ' ########')
    keys = list(data.keys())
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            print('diff between ' + keys[i] + ' and ' + keys[j])
            print(stats.ttest_ind(data[keys[i]], data[keys[j]]))
            print('--------------')